<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> is a general astronomical image toolkit with facilities for reprojection, background matching, coaddition and visualization.  It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.</i>
    
<i>Montage source code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ).  The Python package can be installed from PyPI ("pip install MontagePy"). See
http://montage.ipac.caltech.edu/ for more information.</i>

# MontagePy.main modules: mProjExec

A large percentage of the Montage modules focus on processing a set of images (often retrieved from a mission archive); projecting them to a common frame, adjusting their background levels as a set, and coadding them into a mosaic.

This page is focused on the details of one of these modules: mProjExec.  For a broader context, please see <a href="Mosaic.ipynb">Building a Mosaic with Montage</a> or the <a href="Mosaic_oneshot.ipynb"> one shot </a> version if you just want to see the commands.


<b>Note:</b> The MontagePy python package has no external dependencies. We include other utilities on this page to aid in visualizing MontagePy package results.

In [1]:
from MontagePy.main import mProjExec, mViewer

help(mProjExec)

Help on built-in function mProjExec in module MontagePy.main:

mProjExec(...)
    mProjExec loops over a list of images, reprojecting each of them using mProject, mProjectPP or mProjectQL.
    
    Parameters
    ----------
    path : str
        Path to raw image directory.
    tblfile : str
        Table file list of raw images .
    template : str
        FITS header file used to define the desired output.
    projdir : str, optional
        Path to output projected image directory.
    quickMode : bool, optional
        Flag to force use of mProjectQL function.
    exact : bool, optional
        Flag to force use of mProject over mProjectPP function.  No-op if quickMode is used.
    wholeImages : bool, optional
        Flag to force reprojection of all of each image, even outside template region.
    energyMode : bool, optional
        Pixel values are total energy rather than energy density.
    border : str, optional
        Optional string that contains either a border width or 

## Example

A mosaic image can contain hundreds or thousands of input image, especially from large uniform surveys like 2MASS or SDSS.  Each one must be reprojected to a common frame so mProjExec is a convenience wrapper around the Montage reprojection functions (mProject, mProjectPP, mProjectQL) that simplifies this, looping over an image list an calling the correct function for each one.

Most of the optional arguments to mProjExec are passed-through to the projection function.  In its own right the principle arguments are the path to the input image directory, the list of input images (a metadata table), the output image header template, an output directory, and an ouput table for run statistics.

In [2]:
import os

try:
   os.makedirs('work/M17/projected')
except:
   pass

rtn = mProjExec('M17/raw',
                'M17/rimages.tbl', 
                'M17/M17.hdr', 
                'work/M17/projected')

print(rtn)

{'status': '0', 'count': 85, 'failed': 0, 'nooverlap': 0}


The 'count' shows the number of images reprojected; 'failed' the ones that did not work; and 'nooverlap' the number outside the region of interest.

&nbsp;<p/>

## Error Handling

If mFitExec encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user specifies a table that doesn't exist: 


In [3]:
rtn = mProjExec('M17/raw',
                'M17/unknown.tbl', 
                'M17/region.hdr', 
                'work/M17/projected')

print(rtn)

{'status': '1', 'msg': b'Image metadata file (M17/unknown.tbl) does not exist'}


&nbsp;

# Classic Montage: mProjExec as a Stand-Alone Program

### mProjExec Unix/Windows Command-line Arguments

<p>mProjExec can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mProjExec [-q(uick-mode)][-p rawdir] [-d] [-e(xact)] [-X(whole image)] [-b border] [-r restartrec] [-s statusfile] [-W weightColumn] [-x scaleColumn] images.tbl template.hdr projdir stats.tbl
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mProjExec can be accessed as a library function:</p>


<pre>
/*-***********************************************************************/
/*                                                                       */
/*  mProjExec                                                            */
/*                                                                       */
/*  Montage is a set of general reprojection / coordinate-transform /    */
/*  mosaicking programs.  Any number of input images can be merged into  */
/*  an output FITS file.  The attributes of the input are read from the  */
/*  input files; the attributes of the output are read a combination of  */
/*  the command line and a FITS header template file.                    */
/*                                                                       */
/*  This module, mProjExec, runs one of the reprojection routines        */
/*  (mProject, mProjectPP or mProjectQL) on a set of images to get them  */
/*  all into a common projection.                                        */
/*                                                                       */
/*   char  *path           Path to raw image directory                   */
/*   char  *tblfile        Table file list of raw images                 */
/*   char  *template       FITS header file used to define the desired   */
/*                         output                                        */
/*                                                                       */
/*   char  *projdir        Path to output projected image directory      */
/*   int    quickMode      Flag to force use of mProjectQL function      */
/*   int    exact          Flag to force use of mProject over mProjectPP */
/*                         function.  No-op if quickMode is used.        */
/*                                                                       */
/*   int    expand         Flag to force reprojection of all of each     */
/*                         image, even outside template region           */
/*                                                                       */
/*   int    energyMode     Pixel values are total energy rather than     */
/*                         energy density.                               */
/*                                                                       */
/*   char  *borderStr      Optional string that contains either a border */
/*                         width or comma-separated 'x1,y1,x2,y2, ...'   */
/*                         pairs defining a pixel region polygon where   */
/*                         we keep only the data inside.                 */
/*                                                                       */
/*   char  *scaleCol       Table file can have columns defining scale    */
/*   char  *weightCol      and weighting values to be applied to data    */
/*                         and 'area' output files.                      */
/*                                                                       */
/*   int    restart        Bulk reprojection sometimes needs to be       */
/*                         restarted part way through the list. This is  */
/*                         the record in the table to start with.        */
/*                                                                       */
/*   char  *stats          The programs keeps statistics on each         */
/*                         reprojection in this file.                    */
/*                                                                       */
/*   int    debug          Debugging output level                        */
/*                                                                       */
/*************************************************************************/

struct mProjExecReturn *mProjExec(char *path, char *tblfile, char *template, char *projdir, int quickMode,
                                  int exact, int expand, int energyMode, char *borderStr, char *scaleCol,
                                  char *weightCol, int restart, char *stats, int debugin)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mProjExecReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
   int    count;         // Number of images reprojected
   int    failed;        // Number of reprojections that failed
   int    nooverlap;     // number of images that do not overlap the area of interest
};
</pre>